In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
import matplotlib.pyplot as plt 

In [3]:
df = pd.read_csv('Twitter_Data.csv')

In [4]:
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [5]:
df. isnull().sum()

clean_text    4
category      7
dtype: int64

In [10]:
df.dtypes

clean_text     object
category      float64
dtype: object

In [12]:
X = df['clean_text']
Y = df['category']

In [13]:
X.shape, Y.shape

((162980,), (162980,))

In [14]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, random_state = 42, test_size = 0.3)

In [16]:
xtest.shape, ytest.shape

((48894,), (48894,))